# Regression and Classification in one Model

Goal is to use one model for a regression and a classification. For that I go with the articles from watson.ch again. I want to predict the author of the article (classification) and the number of comments (regression) based on the article. 

## Data

In [1]:
import pandas as pd
#import Python libaries needed for training embbeded vectors 
from gensim.models.word2vec import LineSentence
from gensim.models import Word2Vec
import time # for checking how long the training process takes
import re
import string
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import ipynb.fs.full.Classifier as cl#from https://github.com/ptnplanet/NLTK-Contributions/blob/master/ClassifierBasedGermanTagger/ClassifierBasedGermanTagger.py
import random
% matplotlib inline

In [2]:
# load the dataset
articles = pd.read_csv("C:\\Users\\gwehrm\\Documents\\Repos\\watson_analysis\\watson_schweiz.csv",sep = ";")
articles.head(5)

,title,author,date,nmbr_comments,themes,article
0,Tourismus-Professor pendelt mit Flugzeug zur A...,no_author,"28.03.19, 22:15 28.03.19, 22:40",19,"['Schweiz', 'Gesellschaft & Politik', 'Klima']","['Naaa, wie kommt ihr so zur Uni? Mit dem Fahr..."
1,no_title,no_author,no_date,no_comments,[],['\r\n\t\tMit deiner Anmeldung erklärst du dic...
2,Anstatt mit Bus und Zug fahren mehr Menschen m...,no_author,"28.03.19, 17:39",29,"['Schweiz', 'Gesellschaft & Politik', 'Mobilit...",['\nDer Ausbau des öffentlichen Verkehrs würde...
3,Über 80'000 Franken bei Online-Bank N26 geklau...,no_author,"28.03.19, 17:34",18,"['Digital', 'Schweiz', 'Datenschutz', 'Deutsch...",['\nDie gefeierte Online-Bank N26 verspielt ge...
4,Der Wolf ist zurück – was auch Städter wissen ...,no_author,"28.03.19, 16:19",45,"['Schweiz', 'Wissen', 'Aargau', 'Natur', 'Tier']",['\nDer gesetzliche Schutz des Wolfes wird der...


In [3]:
# filter no_author
data_reduced = articles[-articles['author'].str.contains("no_author")]
# authors_article = data_reduced.groupby('author').count().reset_index()
# for simplicity I will reduce the number of authors. I set a threshold of minimum 50 articles 


# Importing necessary libraries
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
data_reduced["author"] = labelencoder.fit_transform(data_reduced["author"])


g = data_reduced.groupby('author')
data_reduced = g.filter(lambda x: len(x) > 50).reset_index(drop = True)
display(data_reduced.groupby('author').count())

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


,title,date,nmbr_comments,themes,article
author,,,,,
1,63,63,63,63,63
5,113,113,113,113,113
7,149,149,149,149,149
15,104,104,104,104,104
16,152,152,152,152,152
19,155,155,155,155,155
28,52,52,52,52,52
42,133,133,133,133,133
49,99,99,99,99,99


In [4]:
# remove punctuation
exclude = set(string.punctuation)
for index,s in enumerate(data_reduced["article"]):
    exclude = set(string.punctuation)
    data_reduced.loc[index,"article"]= ''.join(ch for ch in s if ch not in exclude)

In [5]:
display(data_reduced.head())

,title,author,date,nmbr_comments,themes,article
0,Zielscheibe der Wut – Tamara Funiciello auf de...,49,"28.03.19, 11:28 28.03.19, 12:40",392,"['Schweiz', 'Facebook', 'Feminismus', 'Gesells...",nTamara Funiciello JusoPräsidentin und Zielsch...
1,Der Hass im Netz nimmt zu – was bedeutet das f...,57,"27.03.19, 10:29 27.03.19, 18:58",26,"['Schweiz', 'Best of watson', 'Facebook', 'Ges...",Die aktuelle Kriminalstatistik zeigt deutlich ...
2,Der Kampf um den Waschturm – ein Zürcher Krie...,57,"26.03.19, 09:26 27.03.19, 01:36",70,"['Schweiz', 'Best of watson', 'Gesellschaft & ...",Tatort Ein Mehrfamilienhaus in Zürich Wipkinge...
3,«Das ist erst der Anfang»: So reagiert die Kli...,1,"25.03.19, 18:42 26.03.19, 11:01",41,"['Schweiz', 'Best of watson', 'Klima', 'Klimas...",Die KlimastreikBewegung feiert mit dem Erdruts...
4,Warum das Klima rockt und Europa nicht: 5 Erke...,42,"25.03.19, 14:53 26.03.19, 06:49",33,"['Schweiz', 'BDP', 'Best of watson', 'CVP', 'F...",nDie Parteipräsidenten Konrad Langhart SVP lin...


In [6]:
#dowloading the stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gwehrm\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
from nltk.corpus import stopwords

In [8]:
# read in the dowloaded corpus 
corp = nltk.corpus.ConllCorpusReader('C:\\Users\\gwehrm\\Documents', 'tiger_release_aug07.corrected.16012013.conll09',
                                     ['ignore', 'words', 'ignore', 'ignore', 'pos'],
                                     encoding='utf-8')

tagged_sents = list(corp.tagged_sents())
random.shuffle(tagged_sents)

# set a split size: use 90% for training, 10% for testing
split_perc = 0.1
split_size = int(len(tagged_sents) * split_perc)
train_sents, test_sents = tagged_sents[split_size:], tagged_sents[:split_size]

# from ClassifierBasedGermanTagger
#train the classifier ()
tagger = cl.ClassifierBasedGermanTagger(train=train_sents)

from germalemma import GermaLemma
lemmatizer = GermaLemma()

accuracy = tagger.evaluate(test_sents)

In [9]:
#the accuracy of the classifier 
display(accuracy)

0.9411487917184822

In [10]:
# in a list comprehension, first tag the word correctly and remove if it is a german stopword
for index,article in enumerate(data_reduced["article"]):
    data_reduced["article"][index]= tagger.tag([word for word in article.split() if word.lower() not in stopwords.words('german')])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
# to get the stem of the word, lemmatization needs to be done for each word
# with thelp of the tagged word, it works

from germalemma import GermaLemma
lemmatizer = GermaLemma()
# passing the word and the POS tag 
for index, tos in enumerate(data_reduced["article"]):
    article_w=[]
    for i in tos:
        try:
            word, N = i
            lemma = lemmatizer.find_lemma(word,N)
            article_w.append(lemma)
        except ValueError:
            continue
    data_reduced.at[index,"article"] = article_w


In [12]:
data_reduced["article"]
# the articles are empty...

0       [ntamara, Funiciello, JusoPräsidentin, Zielsch...
1       [aktuell, Kriminalstatistik, zeigen, deutlich,...
2       [Tatort, Mehrfamilienhaus, Zürich, Wipkingen, ...
3       [KlimastreikBewegung, feiern, Erdrutschsieg, Ö...
4       [Parteipräsident, Konrad, Langhart, SVP, links...
5       [nbesuch, Reitschule, Polizei, Seite, toben, S...
6       [n«i, schänken, Motto, jung, SRFArena, Thema, ...
7       [nsvppräsident, Albert, Rösti, Fraktionschef, ...
8       [Gesellschaft, reich, heißen, glücklich, sagen...
9       [nsbbkund, GA, weiterhin, online, hinterlegen,...
10      [nfranziska, Roth, Wahl, heute, glücklich, Bil...
11      [nbild, KEYSTONE, Wahl, Kanton, Zürich, gelten...
12      [ntschüss, Schweiz, teuer, Ausbildung, müssen,...
13      [Welt, sehen, haben, Mr, Bean, UK, Sagen, bild...
14      [n20, diskutiert, Studio, kündigen, gleich, ba...
15      [Gang, Öffentlichkeit, Stefan, Locher, Beginn,...
16      [nthomas, Ahlburg, neu, Superzug, Stadler, gen...
17      [nzürc